In [1]:
import jax.numpy as jnp
import bayes3d as b
import trimesh
import os
import numpy as np
import trimesh
from tqdm import tqdm


In [4]:

# --- creating the ycb dir from the working directory
bop_ycb_dir = os.path.join(b.utils.get_assets_dir(), "bop/ycbv")

rgbd, gt_ids, gt_poses, masks = b.utils.ycb_loader.get_test_img('50', '640', bop_ycb_dir)


# is ground truth poses for an image gt_poses
print(gt_poses)


mesh_paths = []
offset_poses = []
model_dir = os.path.join(b.utils.get_assets_dir(), "ycb_video_models/models")
for i in tqdm(gt_ids):
    mesh_path = os.path.join(model_dir, b.utils.ycb_loader.MODEL_NAMES[i],"textured.obj")
    _, pose = b.utils.mesh.center_mesh(trimesh.load(mesh_path), return_pose=True)
    print(pose)
    offset_poses.append(pose)
    mesh_paths.append(
        mesh_path
    )

intrinsics = b.Intrinsics(
    rgbd.intrinsics.height, rgbd.intrinsics.width,
    rgbd.intrinsics.fx, rgbd.intrinsics.fx,
    rgbd.intrinsics.width/2, rgbd.intrinsics.height/2,
    rgbd.intrinsics.near, rgbd.intrinsics.far
)

print(intrinsics)

poses = []
for i in range(len(gt_ids)):
    poses.append(
        gt_poses[i] @ b.t3d.inverse_pose(offset_poses[i])
    )
poses = jnp.array(poses)

frames=5
# pose modifications are in object space
poses_m = jnp.array([b.t3d.inverse_pose(b.t3d.transform_from_pos_target_up(
        jnp.array([0.0, 2.0, 0.0]),
        jnp.array([0.0, 0.0, 0.0]),
        jnp.array([0.0, 0.0, 1.0]),
)) @ b.t3d.transform_from_axis_angle(jnp.array([1.0, 0.0, 1.0]), angle) for angle in jnp.linspace(0.0, 4*jnp.pi, frames)])


#need to engineer a better way to get camera trajectories rather than repeatedly rerendering

poses_t = jnp.einsum('ijk,lmj->ilmk',poses_m,poses)
# print(poses[0,...])
# print(poses[-1,...])

poses_t=poses_t[0]
print(poses_t)

# rgbds = render_many(mesh_paths, poses, intrinsics, scaling_factor=1.0, lighting=5.0)
# b.get_rgb_image(rgbds[0].rgb).save("test_ku.png")
# b.get_rgb_image(rgbds[2].rgb).save("test_ku2.png")

[[[ 0.94905835  0.02788146  0.31386635  0.06933129]
  [ 0.11728816  0.8932457  -0.43400028 -0.09377632]
  [-0.29246086  0.44870442  0.8444708   1.025182  ]
  [ 0.          0.          0.          1.        ]]

 [[-0.6896261  -0.7234281  -0.03267294  0.12436341]
  [-0.2806556   0.30858916 -0.9088489  -0.02746224]
  [ 0.6675695  -0.61759585 -0.41584536  0.9460713 ]
  [ 0.          0.          0.          1.        ]]

 [[-0.50055104 -0.86509585 -0.03252145  0.10813791]
  [-0.37298378  0.24940766 -0.89368886 -0.01423071]
  [ 0.781237   -0.43520668 -0.44750842  0.8369227 ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.348734   -0.9341079  -0.07633073  0.00694173]
  [-0.4790473  -0.10765725 -0.87116253  0.07595585]
  [ 0.8055425   0.34037006 -0.48502573  0.8338654 ]
  [ 0.          0.          0.          1.        ]]

 [[-0.64011514  0.76583797  0.06120402 -0.08747824]
  [-0.27533382 -0.3030458   0.9123353   0.03121561]
  [ 0.7172477   0.5671472   0.40484545  0.91173226]
  [ 

 20%|██        | 1/5 [00:02<00:11,  2.94s/it]

[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00 -5.7250075e-04]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00 -1.7089993e-03]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00 -4.8364997e-03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


 40%|████      | 2/5 [00:05<00:08,  2.69s/it]

[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00 -2.6250072e-04]
 [ 0.0000000e+00  1.0000000e+00  0.0000000e+00  1.6014986e-03]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00 -8.3454996e-03]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


...

[[ 1.         0.         0.        -0.0012635]
 [ 0.         1.         0.        -0.002471 ]
 [ 0.         0.         1.         0.011849 ]
 [ 0.         0.         0.         1.       ]]


 80%|████████  | 4/5 [00:10<00:02,  2.63s/it]

[[ 1.         0.         0.         0.0187095]
 [ 0.         1.         0.        -0.012222 ]
 [ 0.         0.         1.         0.0014165]
 [ 0.         0.         0.         1.       ]]


100%|██████████| 5/5 [00:13<00:00,  2.73s/it]

[[ 1.         0.         0.        -0.0090165]
 [ 0.         1.         0.        -0.020988 ]
 [ 0.         0.         1.         0.0021475]
 [ 0.         0.         0.         1.       ]]
Intrinsics(height=480, width=640, fx=1066.778, fy=1066.778, cx=320.0, cy=240.0, near=0.01, far=2.0)
[[[-0.94905835 -0.31386635 -0.02788146  0.699173  ]
  [-0.11728816  0.43400028 -0.8932457  -0.96228224]
  [ 0.29246086 -0.8444708  -0.44870442  2.7188072 ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.6896261   0.03267294  0.7234281   0.05972242]
  [ 0.2806556   0.9088489  -0.30858916 -1.8533127 ]
  [-0.6675695   0.41584536  0.61759585  0.11207449]
  [ 0.          0.          0.          1.        ]]

 [[ 0.50055104  0.03252145  0.86509585  0.04071026]
  [ 0.37298378  0.89368886 -0.24940766 -1.7908741 ]
  [-0.781237    0.44750842  0.43520668 -0.05287987]
  [ 0.          0.          0.          1.        ]]

 [[-0.348734    0.07633073  0.9341079  -0.16355291]
  [ 0.4790473   0.87116253  0

In [5]:
print(gt_poses)

[[[ 0.94905835  0.02788146  0.31386635  0.06933129]
  [ 0.11728816  0.8932457  -0.43400028 -0.09377632]
  [-0.29246086  0.44870442  0.8444708   1.025182  ]
  [ 0.          0.          0.          1.        ]]

 [[-0.6896261  -0.7234281  -0.03267294  0.12436341]
  [-0.2806556   0.30858916 -0.9088489  -0.02746224]
  [ 0.6675695  -0.61759585 -0.41584536  0.9460713 ]
  [ 0.          0.          0.          1.        ]]

 [[-0.50055104 -0.86509585 -0.03252145  0.10813791]
  [-0.37298378  0.24940766 -0.89368886 -0.01423071]
  [ 0.781237   -0.43520668 -0.44750842  0.8369227 ]
  [ 0.          0.          0.          1.        ]]

 [[ 0.348734   -0.9341079  -0.07633073  0.00694173]
  [-0.4790473  -0.10765725 -0.87116253  0.07595585]
  [ 0.8055425   0.34037006 -0.48502573  0.8338654 ]
  [ 0.          0.          0.          1.        ]]

 [[-0.64011514  0.76583797  0.06120402 -0.08747824]
  [-0.27533382 -0.3030458   0.9123353   0.03121561]
  [ 0.7172477   0.5671472   0.40484545  0.91173226]
  [ 

In [6]:
poses.shape

(5, 4, 4)

In [7]:
b.setup_renderer(intrinsics)
model_dir = os.path.join(b.utils.get_assets_dir(),"bop/ycbv/models")
meshes = []
for idx in range(1,22):
    mesh_path = os.path.join(model_dir,"obj_" + "{}".format(idx).rjust(6, '0') + ".ply")
    b.RENDERER.add_mesh_from_file(mesh_path, scaling_factor=10.0/1000.0) 

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (640, 480, 1024)


RuntimeError: Cuda error: 801[cudaGraphicsGLRegisterImage(&s.cudaColorBuffer[i], s.glColorBuffer[i], GL_TEXTURE_3D, cudaGraphicsRegisterFlagsReadOnly);]

In [ ]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [37]:
poses = []
for i in range(len(gt_ids)):
    poses.append(
        gt_poses[i] @ b.t3d.inverse_pose(offset_poses[i])
    )
poses = jnp.array(poses)

for count, i in enumerate(tqdm(gt_ids)):
    mesh_path = os.path.join(model_dir, b.utils.ycb_loader.MODEL_NAMES[i],"textured.obj")
    mesh = trimesh.load(mesh_path)
    b.show_trimesh(str(i), mesh)
    b.show_pose(str(i), poses[count])

100%|██████████| 5/5 [00:10<00:00,  2.07s/it]


In [36]:
b.clear()

In [32]:
for count, i in enumerate(tqdm(gt_ids)):
    pc = jnp.array(b.RENDERER.meshes[i].vertices)
    b.show_cloud(str(i),b.t3d.apply_transform(pc, b.t3d.inverse_pose(gt_poses[count])))

100%|██████████| 5/5 [00:00<00:00, 79.41it/s]


In [28]:
print(gt_poses.shape)

(5, 4, 4)


In [30]:
gt_ids

[1, 3, 4, 9, 14]